# Overview

In this notebook, we conduct data cleaning of the merged table. 

**Author**: Oscar Javier Bastidas Jossa. 

**Email**: oscar.jossa@deusto.es.

In [1]:
import pandas as pd
import numpy as np
from utilities import Data_cleaning

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.express.colors import sample_colorscale
from IPython.display import clear_output

import math
import copy
%matplotlib inline

pd.set_option('display.max_rows', 500)

In [2]:
df = pd.read_hdf('../data/flattened_database_merged_with_session_executions_v02.h5',  key='data')
df.index.names = ["session_execution_id"]
df.replace(np.nan, 0, inplace=True)

In [4]:
# Creating and instance from the Data cleaning to change the type of the datetime column

change_data_type_columns = {'datetime' : ['session_executions_updated_at'],
                            'int_cols': ['session_executions_summary_effort', 'session_executions_summary_value_of_session']}

cleaner = Data_cleaning(df)

# Converting data types of the indicated columns
df = cleaner.converting_data_types(change_data_type_columns)


 --- converting_data_types method executed --- 

converting datetime
converting int_cols


In [20]:
df[[ "user_programs_user_id"]]

,user_programs_user_id
session_execution_id,
4201,640
4070,640
5193,640
5194,640
4191,640
...,...
3659,571
745519,18105
3639,559


In [7]:
# Select those columns to are be useful just with purpose of descrition
columns_description = ['body_parts_focused',
                    'exercises_description_en',
                    'exercises_execution_time',
                    'exercises_joints',
                    'exercises_met_multiplier',
                    'exercises_muscles',
                    'exercises_t1_max',
                    'exercises_t1_min',
                    'exercises_test_correction',
                    'session_executions_created_at',
                    'session_executions_difficulty_feedback',
                    'session_executions_discarded',
                    'session_executions_enjoyment_feedback',
                    'session_executions_imported',
                    'session_executions_reps_executed',
                    'session_executions_user_program_id',
                    'sessions_calories',
                    'sessions_description_en',
                    'sessions_name_en',
                    'sessions_order',
                    'sessions_time_duration',
                    'user_programs_active',
                    'user_programs_completed',
                    'user_programs_program_id',
                    'user_programs_enjoyment',
                    'session_executions_updated_at', 
                    'user_programs_user_id',
                    'users_created_at',
                    'users_updated_at',
                    'users_gender',
                    'users_activity_level',
                    'users_body_type',
                    'users_newsletter_subscription',
                    'users_sign_in_count',
                    'users_notifications_setting',
                    'users_training_days_setting',
                    'users_country',
                    'users_points',
                    'users_scientific_data_usage',
                    'users_best_weekly_streak', 
                    'users_imported',
                    #'session_executions_summary_total_reps',
                    #'session_executions_summary_total_kcal',
                    #'session_executions_summary_effort',
                    #'session_executions_summary_points',
                    #'session_executions_summary_value_of_session',
                    'session_executions_summary_updated_at'
                    ]

df_description = df[columns_description]

# Removing all the columns except the exercises
df_data = df.drop(columns=columns_description)

# Organizing all the columns by the user id
df_description = df_description.sort_values(by=['user_programs_user_id'])

In [8]:
# Getting user id and session_executions_updated_at
df_user_session_execution_date = df_description[['session_executions_updated_at', 'user_programs_user_id']]
users_id = df_user_session_execution_date['user_programs_user_id'].drop_duplicates()

# Organizing dataframe by users and datetime
sorted_rows = []
for user_id in users_id:
    sorted_rows_by_user = df_user_session_execution_date.loc[df_user_session_execution_date['user_programs_user_id'] == user_id].sort_values('session_executions_updated_at')
    sorted_rows.append(sorted_rows_by_user)

# Merging organized concatenated_df with the exercise data
concatenated_df = pd.concat(sorted_rows, ignore_index=False)
df_data = pd.merge(concatenated_df, df_data, left_index=True, right_index=True)

# Replacing NaN with 0
df_data.fillna(0, inplace=True)

# Getting users
users = df_data['user_programs_user_id'].drop_duplicates()

In [9]:
# Faster way to remove the repeated rows (It works well with this dataset, but can generate problems in very few cases with 
# other datasets when sustracting the timestamp from different users and coincidentially they exercise the same day, at the same time (less than 30 seconds))

# # Calculate the difference in seconds between consecutive timestamps
# time_diff = df_data['session_executions_updated_at'].diff().dt.total_seconds()
# 
# #time_diff = time_diff[time_diff > 0] 
# time_diff[time_diff < 0] = 60
# 
# # Include the minuend in the condition by shifting the time difference by one row
# condition = (time_diff < 30) | (time_diff.shift(-1) < 30)
# 
# # Filter rows based on the condition
# filtered_rows = df_data[condition]
# 
# value_counts = filtered_rows['user_programs_user_id'].value_counts()
# 
# # Filter out rows where the value in 'specific_column' occurs less than twice
# filtered_rows = filtered_rows[filtered_rows['user_programs_user_id'].isin(value_counts[value_counts >= 2].index)]
# 
# filtered_rows['user_programs_user_id'].isin(a['user_programs_user_id']).value_counts()

def get_exercises_repeated(df, users):
    """
    Identifies repeated exercises for each user within a DataFrame based on a time difference condition.

    Parameters:
        df (pandas.DataFrame): DataFrame containing exercise data.
        users (list): List of user IDs.

    Returns:
        tuple: Two lists - 
                repeated_exercises: Contain the filtered rows representing repeated exercises for each user.
                last_rows: contain the last row of filtered data for each user.
    """
    repeated_exercises = []
    last_rows = []
    for user in users:
        
        user_data  = df.loc[df['user_programs_user_id'] == user]

        # Calculate the difference in seconds between consecutive timestamps
        time_diff = user_data['session_executions_updated_at'].diff().dt.total_seconds()

        # Include the minuend in the condition by shifting the time difference by one row
        condition = (time_diff < 10) | (time_diff.shift(-1) < 10)

        # Filter rows based on the condition
        filtered_rows = user_data[condition]

        # Select the last row of filtered data
        last_row = filtered_rows.iloc[-1:]

        # Append filtered rows and last row to respective lists
        repeated_exercises.append(filtered_rows)
        last_rows.append(last_row)

    return repeated_exercises, last_rows


list_repeated_exercises, list_last_rows = get_exercises_repeated(df_data, users)

# Create the dataframes of the repeated_exercises and the list of last rows
repeated_exercises = pd.concat(list_repeated_exercises, ignore_index=False)
list_last_rows = pd.concat(list_last_rows, ignore_index=False)

# Droping the last rows form the repeated exercises
repeated_exercises_last_rows_substracted = repeated_exercises.drop(list_last_rows.index)

# Getting the final dataframe after removing the the session executions duplicated with a difference less than 10 seconds 
df_data_remove_duplicated = df_data.drop(repeated_exercises_last_rows_substracted.index)

print("df with rows repeated: ", df_data.shape)
print("df of rows repeated", repeated_exercises.shape)
print("last rows from repeated_exercises", list_last_rows.shape)
print("difference from repeated_exercises - list_last_rows", repeated_exercises_last_rows_substracted.shape), 
print("df with wrong data removed: ", df_data_remove_duplicated.shape)
print("Final check ", df_data.shape[0] - df_data_remove_duplicated.shape[0])

df with rows repeated:  (51355, 9141)
df of rows repeated (2150, 9141)
last rows from repeated_exercises (269, 9141)
difference from repeated_exercises - list_last_rows (1881, 9141)
df with wrong data removed:  (49474, 9141)
Final check  1881


In [10]:
# This dataset is tested when the session executions with less than 10 seconds have been removed
df_data_remove_duplicated.to_hdf('../data/flattened_database_merged_with_session_executions_v02.h5', key='data', mode='w')

In [15]:

def plot_exercises(df_plot, user):
    
    """
    Plots exercises for a specific user from a DataFrame and show them in subplots

    Parameters:
        df_plot (pandas.DataFrame): DataFrame containing exercise data.
        user (str): User ID for whom exercises are to be plotted.
    """

    # Getting only the exercises
    exercises = df_plot.columns.drop(['session_executions_updated_at', 'user_programs_user_id'])
    
    # Configuring the parameters for the plot 
    len_plot = math.ceil(len(exercises)/2)
    fig = make_subplots(rows=len_plot, cols=2, subplot_titles=exercises)
    row_count = 1
    col_count = 1
    x_color = np.linspace(0, 1, 25)
    c_color = sample_colorscale('jet', list(x_color))
    count_color = 0

    # For to plot all the exercises of the user
    for index, exercise in enumerate(exercises):
        fig.add_trace(
        go.Scatter(y=df_plot[exercise], x =df_plot['session_executions_updated_at'], 
                mode="lines+markers",
                name= exercise,
                marker=dict(
                        color=c_color[count_color],
                        size=5)),
                row=row_count, col=col_count)
        
        if (index%2 != 0):
            row_count = row_count + 1
            col_count = 1
        else:
            col_count = col_count + 1
        
        count_color = count_color + 1
        if count_color == 25:
            count_color = 0
    
    fig.update_layout(height=len_plot*400, width=1500, title_text="Exercises per User {}".format(user))

    # Clear previous output
    clear_output(wait=True)

    fig.show()

In [16]:
def plot_all_exercises(df_plot, user):
    """
    Plots all exercises for a specific user from a DataFrame and show them in a single graph

    Parameters:
        df_plot (pandas.DataFrame): DataFrame containing exercise data.
        user (str): User ID for whom exercises are to be plotted.
    """    
    
    # Extract variable names
    value_vars = df_plot.columns.drop(['session_executions_updated_at', 'user_programs_user_id'])
    
    # Melt the DataFrame
    df_melt = pd.melt(df_plot, id_vars=['session_executions_updated_at', 'user_programs_user_id'], value_vars=value_vars)
    
    fig = px.scatter(df_melt, x="session_executions_updated_at", y="value", color="variable", title="Exercises per User {}".format(user))
    
    # Clear previous output
    clear_output(wait=True)
    
    fig.show()

In [17]:
for user in users:
    
    user_data  = df_data_remove_duplicated.loc[df_data['user_programs_user_id'] == user]
    print("User number : ", user)
    while True:
        next_action = input("Next action: ")
        if next_action == 'y':
            break
        if next_action == 'c':
            break
        else:
            pass
    if next_action == 'c':
        break

    series_exec = []

    for index, row in user_data.iterrows():
        series_number = row[row.astype(bool)]
        series_number = series_number[~series_number.keys().str.contains('_time_')]
        series_number = series_number[~series_number.keys().str.contains('_reps_')]
        series_exec.append(series_number.to_dict())

    df_plot = pd.DataFrame(series_exec)
    #plot_exercises(df_plot, user)
    plot_all_exercises(df_plot, user)
    

User number :  108


In [18]:
# Extract date from 'session_executions_updated_at'
df_data_remove_duplicated['date'] = df_data_remove_duplicated['session_executions_updated_at'].dt.date

# Group by 'user_programs_user_id' and 'date', then count the number of sessions for each group
sessions_per_day = df_data_remove_duplicated.groupby(['user_programs_user_id', 'date']).size()

# Filter out users who executed more than one session per day
users_with_more_than_one_session = sessions_per_day[sessions_per_day > 1]

# calculate the number of rows deleted (not sure it this is ok! (it is just to test))
rows_with_the_same_day_deleted = users_with_more_than_one_session.values.sum() - users_with_more_than_one_session.value_counts().sum()

print(rows_with_the_same_day_deleted)

df_data_remove_duplicated.shape[0] - rows_with_the_same_day_deleted

3282


46192

In [15]:
'''
import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

datetime1 = datetime.datetime(2021,10,30, 16)
datetime2 = datetime.datetime(2021,10,30, 18)
df_before_sum_per_day = df_data_remove_duplicated.loc[(df_data_remove_duplicated['user_programs_user_id'] == 108) &
                                  (df_data_remove_duplicated['session_executions_updated_at'] > datetime1) &
                                  (df_data_remove_duplicated['session_executions_updated_at'] < datetime2) ]

df_before_sum_per_day.loc[:, (df_before_sum_per_day != 0).any(axis=0)]
df_before_sum_per_day.to_hdf('../data/test/df_test_user108.h5', key='data', mode='w')
'''
# The previous code was very useful to test those users which the total_reps does not correspond with
# the number of session_executions_summary_total_reps.
# The df_test_user108 have to be tested in the data_cleaning_dataframe_session_v0.ipynb file

"\nimport datetime\n\npd.set_option('display.max_rows', 500)\npd.set_option('display.max_columns', 500)\n\ndatetime1 = datetime.datetime(2021,10,30, 16)\ndatetime2 = datetime.datetime(2021,10,30, 18)\ndf_before_sum_per_day = df_data_remove_duplicated.loc[(df_data_remove_duplicated['user_programs_user_id'] == 108) &\n                                  (df_data_remove_duplicated['session_executions_updated_at'] > datetime1) &\n                                  (df_data_remove_duplicated['session_executions_updated_at'] < datetime2) ]\n\ndf_before_sum_per_day.loc[:, (df_before_sum_per_day != 0).any(axis=0)]\ndf_before_sum_per_day.to_hdf('../data/test/df_test_user108.h5', key='data', mode='w')\n"

The "rows_with_the_same_day_deleted" above should be the same after apply the operaration below that sum all the exercises grouped by user id and date

In [26]:
df_data_remove_duplicated.shape

(49474, 9142)

In [27]:
# Assuming df is your DataFrame

# Convert 'session_executions_updated_at' column to datetime type
df_data_remove_duplicated['session_executions_updated_at'] = pd.to_datetime(df_data_remove_duplicated['session_executions_updated_at'])

# Extract columns to sum
columns_to_sum = df_data_remove_duplicated.columns.drop(['session_executions_updated_at', 'user_programs_user_id', 'date'])

# Group by 'user_programs_user_id' and 'date', then calculate sum for '1 leg bridge (left)' for each group
df_data_remove_duplicated[columns_to_sum] = df_data_remove_duplicated.groupby(['user_programs_user_id', 'date'])[columns_to_sum].transform('sum')

# Drop duplicate rows based on 'user_programs_user_id' and 'date'
df_data_remove_duplicated = df_data_remove_duplicated.drop_duplicates(subset=['user_programs_user_id', 'date'])

df_final = copy.copy(df_data_remove_duplicated)

The df_final containts the sum of the sessions per day for each user. It is organized per user, and then per consecutive days. For this dataset also was correct the sessions that were multiple times repeated in short periods of time

In [18]:
df_final.shape

(46192, 9140)

In [29]:
# This dataset is after removing the wrong session executions (duplicated with less than second of diference) and after grouping all the session executions per day for each user
df_final.to_hdf('../data/cleaned_dataset_v1.h5', key='data', mode='w')
df_final

C:\Users\eVida-PC\AppData\Local\Temp\ipykernel_17824\2631874977.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->date,key->block3_values] [items->Index(['date'], dtype='object')]

  df_final.to_hdf('../data/cleaned_dataset_v1.h5', key='data', mode='w')


,session_executions_updated_at,user_programs_user_id,1 leg bridge (left),1 leg bridge (left)_reps_1,1 leg bridge (left)_reps_10,1 leg bridge (left)_reps_11,1 leg bridge (left)_reps_12,1 leg bridge (left)_reps_13,1 leg bridge (left)_reps_14,1 leg bridge (left)_reps_15,...,Zancadas_reps_3,Zancadas_time_1,Zancadas_time_2,Zancadas_time_3,session_executions_summary_effort,session_executions_summary_points,session_executions_summary_total_kcal,session_executions_summary_total_reps,session_executions_summary_value_of_session,date
session_execution_id,,,,,,,,,,,,,,,,,,,,,
3737,2021-06-11 18:00:35.640406,108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,100,2190,330,3,2021-06-11
5830,2021-10-30 16:51:09.117908,108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8,200,136,250,6,2021-10-30
17048,2021-12-08 19:19:10.266601,108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7,100,272,576,4,2021-12-08
17351,2021-12-09 19:13:22.047997,108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,100,68,150,2,2021-12-09
715926,2022-01-24 11:38:17.767057,108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10,200,204,280,6,2022-01-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746183,2022-05-25 20:36:55.438881,18157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5,100,98,107,3,2022-05-25
746178,2022-05-25 20:12:56.904269,18165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5,100,66,274,3,2022-05-25
746199,2022-05-26 00:49:19.809268,18165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5,100,132,276,3,2022-05-26


In [119]:
# To make the comparisson run again the where the "df_data_remove_duplicated" is obtained
# 1.) Check the users to compare
# print(df_data_remove_duplicated.shape, df_final.shape)
# users_with_more_than_one_session.head(100)

# 2.) select the users to compare
## sub_dataframe_1 = df_data_remove_duplicated.loc[df_data_remove_duplicated['user_programs_user_id'] == 620] # Change this number according the user to compare
## sub_dataframe_2 = df_final.loc[df_final['user_programs_user_id'] == 620] # Change this number according the user to compare

# 3) Select the dates to compare
### date1_sub_dataframe_1 = sub_dataframe_1.loc[[714164]] # Change this index according the First session_executions_updated_at_ to compare
### date2_sub_dataframe_1 = sub_dataframe_1.loc[[714202]] # Change this index according the Second session_executions_updated_at_ to compare
### date3_sub_dataframe_1 = sub_dataframe_1.loc[[714214]] # Change this index according the Third session_executions_updated_at_ to compare
### date_exercises_summed_sub_dataframe_2 = sub_dataframe_2.loc[[714164]] # # Change this index according the First session_executions_updated_at_ to compare in the second df

#### sub_dataframe_1.loc[714164:714214,  (date1_sub_dataframe_1.all()) | (date2_sub_dataframe_1.all()) | (date3_sub_dataframe_1.all())] # Change the number according the previous indexes of the session_executions_updated_at_ to compare
#### date_exercises_summed_sub_dataframe_2.loc[:, date_exercises_summed_sub_dataframe_2.all()]